In [1]:
import numpy as np
import time
import tigramite
from tigramite.pcmci import PCMCI
from tigramite import data_processing as pp
from tigramite.independence_tests.parcorr import ParCorr

In [2]:
PROJ_ROOT = '/Users/mloui/Documents/STAR/SOFAR'

In [3]:
# Quickscat
all_data = np.loadtxt(f'{PROJ_ROOT}/data/NASA_R3/raw_telemetry/quickscat/fault_quickscat.csv',
                 delimiter=",", dtype=str)
print(all_data.shape)

times = all_data[1:,0]
var_names = all_data[0,1:]
data = all_data[1:,1:].astype(float)

print(times.shape)
print(var_names.shape)
print(data.shape)

(104545, 11)
(104544,)
(10,)
(104544, 10)


In [42]:
# KSP Orbital Satellite
all_data = np.loadtxt(f'{PROJ_ROOT}/data/NASA_R3/raw_telemetry/ksp_orbital_satellite/errors/Pressure_Caused_Navigation_SEU.csv',
                 delimiter=",", dtype=str)
#all_data = all_data[:,:-1] #do this for no_errors/Orbit_No_Errors_Chunk_1.csv
print(all_data.shape)

times = all_data[1:,0]
var_names = all_data[0,1:]
data = all_data[1:,1:].astype(float)

print(times.shape)
print(var_names.shape)
print(data.shape)

(759, 17)
(758,)
(16,)
(758, 16)


In [4]:
# Make smaller subset of data if desired
print(times.shape)
print(var_names.shape)
print(data.shape)
small_times = times[:3000]
small_data = data[:3000,:]
print(small_times.shape)
print(small_data.shape)

(104544,)
(10,)
(104544, 10)
(3000,)
(3000, 10)


In [5]:
# Create dataframe and pcmci object
dataframe = pp.DataFrame(small_data, 
                         datatime=small_times, 
                         var_names=var_names)

parcorr = ParCorr(significance='analytic')

pcmci = PCMCI(dataframe=dataframe, 
              cond_ind_test=parcorr,
              verbosity=0)

In [6]:
# Run once
results = pcmci.run_pcmci(pc_alpha=0.01, tau_max=1)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [84]:
# Get average runtime
num_runs = 50
total = 0
for i in range(num_runs):
    start = time.time()
    results = pcmci.run_pcmci(pc_alpha=0.01, tau_max=3)
    end = time.time()
    total+=(end - start)
avg_time = total/num_runs
print(avg_time)

0.26195130348205564


In [7]:
graph = results['graph']
val_matrix = results['val_matrix']
p_matrix = results['p_matrix']

print(graph.shape)
print(val_matrix.shape)
print(p_matrix.shape)

(10, 10, 2)
(10, 10, 2)
(10, 10, 2)


In [8]:
# Get the most correlated mnemonic for each mnemonic
tau = 1
tau_vals = val_matrix[:,:,tau]
print(tau_vals.shape)
max_corr = np.argmax(tau_vals, axis=0)
print(max_corr.shape)

for i in range(len(tau_vals)):
    print(f'{var_names[max_corr[i]]} --> {var_names[i]}')

(10, 10)
(10,)
TCPV6T_(C) --> TCPV6T_(C)
PWBUSV_(V)_min --> PWBUSV_(V)
PWBUSV_(V)_min --> PWBUSV_(V)_min
PWBUSV_(V)_min --> PWBUSV_(V)_max
PWBUSV_(V)_count --> PWBUSV_(V)_count
TRW1MT_(C)_min --> TRW1MT_(C)
TRW1MT_(C)_min --> TRW1MT_(C)_min
TRW1MT_(C) --> TRW1MT_(C)_max
PWBUSV_(V) --> TRW1MT_(C)_count
PWBUSI_(A) --> PWBUSI_(A)
